In [1]:
import base64
import datetime
import pandas as pd
import json
from google.cloud import bigquery

client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: research-311404


In [2]:
## process the assignee dataset
import pandas as pd
import numpy as np
assignee= pd.read_csv('g_assignee_disambiguated.tsv', sep='\t')
assignee.head()

,patent_id,assignee_sequence,assignee_id,disambig_assignee_individual_name_first,disambig_assignee_individual_name_last,disambig_assignee_organization,assignee_type,location_id
0,4488683,0,b942050d-150f-42e4-83d4-a7cde0870f82,NaN,NaN,Metal Works Ramat David,3.0,50dc5d46-16c8-11ed-9b5f-1234bde3cd05
1,5856666,0,dd8cd1db-a0f0-4b3b-ba4a-25038814e332,NaN,NaN,U.S. Philips Corporation,2.0,92237ca2-16c8-11ed-9b5f-1234bde3cd05
2,5204210,0,8d18afca-9c87-4e93-acd9-db539730dc3b,NaN,NaN,Xerox Corporation,2.0,0cd1998f-16c8-11ed-9b5f-1234bde3cd05
3,5302149,1,add68f05-55da-4cd8-9c3a-85d2cf1e1798,NaN,NaN,COMMONWEALTH SCIENTIFIC AND INDUSTRIAL RESEARC...,3.0,4d36742f-16c8-11ed-9b5f-1234bde3cd05
4,D397841,0,a8173451-b1cb-4071-bf4f-b844df2703a7,NaN,NaN,adidas AG,3.0,280c9de4-16c8-11ed-9b5f-1234bde3cd05


In [5]:
assignee = assignee[['patent_id','assignee_type']]
dataset = client.get_dataset('research-311404.temp')  # API request
table_ref = dataset.table('assignee')
job = client.load_table_from_dataframe(assignee, table_ref, location="US")
job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/research-311404/datasets/temp/tables/assignee


classification of assignee (1 - Unassigned, 2 - US Company or Corporation, 3 - Foreign Company or Corporation, 4 - US Individual, 5 - Foreign Individual, 6 - US Federal Government, 7 - Foreign Government, 8 - US County Government, 9 - US State Government. Note: A "1" appearing before any of these codes signifies part interest

In [2]:
sql = """
        CREATE TABLE research-311404.patent_GNN_Data.assignee_full_data
        AS 
        SELECT patent_id, MAX(CASE WHEN assignee_type = 2 OR assignee_type = 3 OR
        assignee_type = 12 OR assignee_type = 13 
        THEN 1 ELSE 0 END) AS with_company,
        MAX(CASE WHEN assignee_type = 4 OR assignee_type = 5
            OR assignee_type = 14 OR assignee_type = 15 THEN 1 ELSE 0 END) AS individual,
        MAX(CASE WHEN assignee_type = 6 OR assignee_type = 7 OR assignee_type = 8 OR assignee_type = 9
            OR assignee_type = 16 OR assignee_type = 17 OR assignee_type = 18 OR assignee_type = 19
        THEN 1 ELSE 0 END) AS with_government
        FROM `research-311404.temp.assignee` GROUP BY patent_id
        
        """

job = client.query(sql)  # API request.
job.result()  # Waits for the query to finish.
table_id = "research-311404.patent_GNN_Data.assignee_full_data"
table = client.get_table(table_id)
print("Table {} now contains {} rows".format(table_id, table.num_rows))

Table research-311404.patent_GNN_Data.assignee_full_data now contains 7572670 rows
